In [28]:
import numpy as np
import scipy

We create two large matrices with random complex entries.

In [3]:
N = 100
A = np.random.rand(N,N) + 1j*np.random.rand(N,N)
B = np.random.rand(N,N) + 1j*np.random.rand(N,N)

We want to calculate $C = AB$. 

## Approach 1: Explicit matrix-matrix product

In [4]:
def func1(A,B):
    C = np.zeros((N,N),dtype=complex)
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i,j] += A[i,k]*B[k,j]
    return C

## Approach 2: Use Numpy's array multiplication.

In [5]:
def func2(A,B):
    return np.matmul(A,B)

## Approach 3: Use naive Einsum.

In [6]:
def func3(A,B):
    return np.einsum('ik,kj->ij',A,B, optimize=False)

## Approach 4: Use optimized Einsum

In [7]:
def func4(A,B):
    return np.einsum('ik,kj->ij',A,B, optimize=True)

## Approach 5: Use wrapper for BLAS 3 routine

In [8]:
def func5(A,B):
    return scipy.linalg.blas.zgemm(A,B)

Now we can compare some timings

In [10]:
%timeit func1(A,B) # Explicit loops

153 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit func2(A,B) # Numpy's

31.9 μs ± 2.68 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%timeit func3(A,B) # Naive einsum

1.18 ms ± 3.53 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%timeit func4(A,B) # Optimized einsum

57.2 μs ± 1.23 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%timeit func4(A,B) #Wrapper for BLAS 3

59.3 μs ± 771 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


All functions are calculating the same quantity, however not all of them are **optimized**!!

Some optimizations are very technical and related to how the code is written to handle arrays in the memory. Others are much more straightforward and can/should be done at the level of our code. 

Now we create a matrix that has a clear structure.

In [15]:
A = np.diag(-2*np.ones((N)), 0) + np.diag(np.ones((N-1)), 1) + np.diag(np.ones((N-1)), -1)
A[0,-1] = 1
A[-1,0] = 1

In [16]:
print(A)

[[-2.  1.  0. ...  0.  0.  1.]
 [ 1. -2.  1. ...  0.  0.  0.]
 [ 0.  1. -2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -2.  1.  0.]
 [ 0.  0.  0. ...  1. -2.  1.]
 [ 1.  0.  0. ...  0.  1. -2.]]


How can we efficiently multiply this matrix with a vector?

In [25]:
b = np.random.rand(N)

## Approach 1: Explicit Matrix-vector product 

In [18]:
def func1(A,x):
    y = np.zeros((N,1))
    for i in range(N):
        for j in range(N):
            y[i] += A[i,j]*x[j]
    return y

## Approach 2: Exploit the sparsity pattern.

In [19]:
def func2(x):
    y = np.zeros((N,1))
    i1 = np.roll(range(N),1)
    i2 = np.roll(range(N),-1)
    for i in range(N):
        y[i] = -2*x[i] + x[i1[i]] + x[i2[i]]
    return y

## Approach 3: Use optimized Einsum.

In [20]:
def func3(A,x):
    return np.einsum('ij,j->i',A,x)

## Approach 4: Wrapper for BLAS 2

In [30]:
def func4(A,x):
    return scipy.linalg.blas.zgemv(1.0,A,x)

In [22]:
%timeit func1(A,b) # Explicit loops

8.28 ms ± 17 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit func2(b) # Implicit action

107 μs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%timeit func3(A,b) # Optimized einsum

2 μs ± 2.58 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%timeit func4(A,b) # Wrapper for BLAS 2

22.7 μs ± 1.69 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Try larger $N$:

In [61]:
N = 1000

In [76]:
A = np.diag(-2*np.ones((N)), 0) + np.diag(np.ones((N-1)), 1) + np.diag(np.ones((N-1)), -1)
A[0,-1] = 1
A[-1,0] = 1
b = np.random.rand(N)

In [77]:
%timeit func2(b)

391 μs ± 2.65 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [79]:
%timeit func3(A,b)

174 μs ± 61.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


The performance of our optimized explicit loops starts getting close to the background-optimized function that does not exploit sparsity!

**Why not always use Numpy's matmul()?**

Memory limitations!! We cannot store the matrices when they are too large, e.g. $N= 10^7$.

But in our second example that is not a problem because the matrix is very sparse and structured, so it is very easy to write a function which returns its action on a vector.

** Why should we care about this type of performance? **

**Many** problems can be formulated in terms of linear systems of the form $Ax = b$, where $A$ is an extremely large matrix and often times very sparse. We cannot afford to explicitly build and store $A$ to calculate $A^{-1}$ via Gaussian-Elimination!!

We focus not on finding $A^{-1}$, but on finding $A^{-1}b$.

Many **iterative methods** approximate $A^{-1}b$ as a polynomial of $P(A)$ acting on $b$, which is a very good approximation!

$A^{-1}b \approx a_0 b + a_1 Ab + a_2 A^2 b + a_3 A^3 b + ...$

We need the operation of acting with A on a vector to be **very** efficient!!